In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
from tensorflow.keras.optimizers import Adam
import os
import cv2
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
import os
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
batch_size = 32
img_size=224
epochs=10
NUM_CLASSES=2

# Set the paths to your train and test folders
train_folder = '/kaggle/input/malayakew-plant-leaf-dataset/MK/D1/train'
test_folder = '/kaggle/input/malayakew-plant-leaf-dataset/MK/D1/test'

selected_classes = ['Class (1)', 'Class (10)']

In [ ]:
# Function to load and preprocess images for selected classes
def load_and_preprocess_data(folder_path, selected_classes):
    data = []
    labels = []
    for label, class_name in enumerate(selected_classes):
        class_path = os.path.join(folder_path, class_name)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  # Adjust the size according to your requirements
            data.append(img)
            labels.append(label)
    return np.array(data), np.array(labels)

In [ ]:
# Load images from train folder for selected classes
train_data, train_labels = load_and_preprocess_data('/kaggle/input/malayakew-plant-leaf-dataset/MK/D1/train', selected_classes)

# Load images from test folder for selected classes
test_data, test_labels = load_and_preprocess_data('/kaggle/input/malayakew-plant-leaf-dataset/MK/D1/test', selected_classes)

In [ ]:
# Convert OpenCV images to NumPy array
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Apply augmentation only on training data
datagen.fit(train_data)

In [ ]:
# Split data into train and validation sets
x_train, val_data, y_train, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [ ]:
print(x_train.shape)
print(val_data.shape)
print(y_train.shape)
print(val_labels.shape)

In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Build the model with Dropout layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model with validation_split and class weights
history = model.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(val_data, val_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Predictions on validation set
y_pred = model.predict(val_data).round()

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision, Recall, F1 Score
precision, recall, f1_score, _ = precision_recall_fscore_support(val_labels, y_pred, average='binary')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

# Classification Report
class_report = classification_report(val_labels, y_pred)
print("Classification Report:")
print(class_report)

In [ ]:
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r',label="Training accuracy")
plt.plot(epochs,val_acc,'b',label="Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# Create VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model
model2 = Sequential()
model2.add(vgg16_model)
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model2.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

In [ ]:
test_loss, test_accuracy = model2.evaluate(val_data, val_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Predictions on validation set
y_pred = model2.predict(val_data).round()

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision, Recall, F1 Score
precision, recall, f1_score, _ = precision_recall_fscore_support(val_labels, y_pred, average='binary')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

# Classification Report
class_report = classification_report(val_labels, y_pred)
print("Classification Report:")
print(class_report)

In [ ]:
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r',label="Training accuracy")
plt.plot(epochs,val_acc,'b',label="Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
InceptionV3_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model
model3 = Sequential()
model3.add(vgg16_model)
model3.add(Flatten())
model3.add(Dense(256, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

# Compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model3.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

In [ ]:
test_loss, test_accuracy = model3.evaluate(val_data, val_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Predictions on validation set
y_pred = model3.predict(val_data).round()

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision, Recall, F1 Score
precision, recall, f1_score, _ = precision_recall_fscore_support(val_labels, y_pred, average='binary')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

# Classification Report
class_report = classification_report(val_labels, y_pred)
print("Classification Report:")
print(class_report)

In [ ]:
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r',label="Training accuracy")
plt.plot(epochs,val_acc,'b',label="Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
MobileNetV2_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model
model4 = Sequential()
model4.add(vgg16_model)
model4.add(Flatten())
model4.add(Dense(256, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))

# Compile the model
model4.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model4.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(val_data, val_labels),
    callbacks=[early_stopping]
)

In [ ]:
test_loss, test_accuracy = model4.evaluate(val_data, val_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Predictions on validation set
y_pred = model4.predict(val_data).round()

# Confusion Matrix
conf_matrix = confusion_matrix(val_labels, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Precision, Recall, F1 Score
precision, recall, f1_score, _ = precision_recall_fscore_support(val_labels, y_pred, average='binary')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1_score:.4f}')

# Classification Report
class_report = classification_report(val_labels, y_pred)
print("Classification Report:")
print(class_report)

In [ ]:
acc = history.history['accuracy']
val_acc= history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs,acc,'r',label="Training accuracy")
plt.plot(epochs,val_acc,'b',label="Validation accuracy")
plt.title('Training and Validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.title('Training and Validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()